In [1]:
from sklearn import datasets
import pandas as pd

In [2]:
iris = datasets.load_iris()

In [9]:
df = pd.DataFrame(iris.data)
df.columns = ["sl", "sw", 'pl', 'pw']

In [10]:
#Function to find label for a value
#if MIN_Value <=val < (m + Mean_Value) / 2 then it is assigned label a
#if (m + Mean_Value) <=val < Mean_Value then it is assigned label b
#if (Mean_Value) <=val < (Mean_Value + MAX_Value)/2 then it is assigned label c
#if (Mean_Value + MAX_Value)/2 <=val <= MAX_Value  then it is assigned label d

def label(val, *boundaries):
    if (val < boundaries[0]):
        return 'a'
    elif (val < boundaries[1]):
        return 'b'
    elif (val < boundaries[2]):
        return 'c'
    else:
        return 'd'

#Function to convert a continuous data into labelled data
#There are 4 lables  - a, b, c, d
def toLabel(df, old_feature_name):
    second = df[old_feature_name].mean()
    minimum = df[old_feature_name].min()
    first = (minimum + second)/2
    maximum = df[old_feature_name].max()
    third = (maximum + second)/2
    return df[old_feature_name].apply(label, args= (first, second, third))

In [12]:
#Convert all columns to labelled data
df['sl_labeled'] = toLabel(df, 'sl')
df['sw_labeled'] = toLabel(df, 'sw')
df['pl_labeled'] = toLabel(df, 'pl')
df['pw_labeled'] = toLabel(df, 'pw')
df

,sl,sw,pl,pw,sl_labeled,sw_labeled,pl_labeled,pw_labeled
0,5.1,3.5,1.4,0.2,b,c,a,a
1,4.9,3.0,1.4,0.2,a,b,a,a
2,4.7,3.2,1.3,0.2,a,c,a,a
3,4.6,3.1,1.5,0.2,a,c,a,a
4,5.0,3.6,1.4,0.2,a,c,a,a
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,c,b,c,d
146,6.3,2.5,5.0,1.9,c,a,c,d
147,6.5,3.0,5.2,2.0,c,b,c,d
148,6.2,3.4,5.4,2.3,c,c,d,d


In [13]:
df.drop(['sl', 'sw', 'pl', 'pw'], axis = 1, inplace = True)

In [14]:
set(df['sl_labeled'])

{'a', 'b', 'c', 'd'}

In [154]:
import math as m
def entropy(arr):
    ans = 0
    total = 0
    
    for i in range(len(arr)):
        total +=arr[i]
    for i in range(len(arr)):
        if arr[i]!=0:
            ans += (arr[i]/total)*(m.log(arr[i]/total))
    
    if(ans==0.0):        
        return 0.0
    else:
        return ans*-1

In [166]:
def split_info(arr):
    total = 0
    for i in range(len(arr)):
        total +=arr[i]
    
    ans = 0
    for i in range(len(arr)):
        if arr[i]!=0:
            ans += (arr[i]/total)*(m.log(arr[i]/total))
            
    #using an if else here because otherwise it prints -0 when the ans is 0
    if(ans==0.0):        
        return 0.0
    else:
        return ans*-1

In [173]:
#defining tree node
class tree_node:
    def __init__(self,count_0,count_1,count_2):
        self.a = None
        self.b = None        
        self.c = None        
        self.d = None
        self.no_of_0 = count_0
        self.no_of_1 = count_1
        self.no_of_2 = count_2             

In [204]:
#Using for printing the tree
def print_tree(root,cnt):
    if root == None:
        return 
    print("No. of zeroes at Level ",cnt,"are =",root.no_of_0)
    print("No. of ones at Level ",cnt,"are =",root.no_of_1)
    print("No. of two's at Level ",cnt,"are =",root.no_of_2)
    print("")
    
    print_tree(root.a,cnt+1)
    print_tree(root.b,cnt+1)    
    print_tree(root.c,cnt+1)    
    print_tree(root.d,cnt+1)    

In [212]:
top_node = None
def build_tree(df, y, unused_features,count):
    
    best_feature = "" #the best feature that we will get after running the loop
    least_mistake=1e18 #the best feature is the one with least mistake
    
    #the count of all the elements at current i.e before spliiting
    count_0_b = 0
    count_1_b = 0
    count_2_b = 0
    
    #this loop calculates the count of 0,1 and 2 for the current level
    for  i in range(len(y)):
        if y.iloc[i,0] == 0:
            count_0_b+=1
        elif y.iloc[i,0] == 1:
            count_1_b+=1
        elif y.iloc[i,0] ==2:
            count_2_b+=1
   
    #printing what was required in the question
    print("LEVEL ",count)
    print("count of 0 = ",count_0_b)
    print("count of 1 = ",count_1_b)
    print("count of 2 = ",count_2_b)
    
    #This is for tree implementation.We are building a new treenode here.
    curr_node = tree_node(count_0_b,count_1_b,count_2_b)
    
    
    #Now we are storing the counts of 0,1,2 in an array for better calculations(using the function declared above)
    arr = []
    arr.append(count_0_b)
    arr.append(count_1_b)
    arr.append(count_2_b)
    
    #calculating th current entropy
    curr_entropy = entropy(arr)
    
    #printing it
    print("Current Entropy is =",curr_entropy)
    
    #The base case(where if further splitting is not possible then we will return from the current function)
    if len(set(y[0]))==1 or len(unused_features) == 0:
        print("Reached leaf node")
        print("")
        return curr_node
    
    #Now we are iterating over the unused features to find out the best feature
    for f in unused_features:
        possible_values = set(df[f])
        
        #mistake_count while predicting for the current feature
        mistake = 0
        #now iteratin over the different possible values in the selected feature
        for val in possible_values:
            #the required y that has rows corresponding to the curr val in the feature
            y_reqd = y[df[f]==val]
            
            #calculating the count of 0,1 and 2 for the particaular value in the currernt feature
            count_0 = 0
            count_1 = 0
            count_2 = 0
            for  i in range(len(y_reqd)):
                if y_reqd.iloc[i,0] == 0:
                    count_0+=1
                elif y_reqd.iloc[i,0] == 1:
                    count_1+=1
                elif y_reqd.iloc[i,0] ==2:
                    count_2+=1
                
            #prediction will be the max of the 3 counts
            ans = max(count_0,count_1,count_2)
            final_pred = 0
            if ans == count_0:
                final_pred = 0
                
            elif ans==count_1:
                final_pred = 1
                
            elif ans == count_2:
                final_pred = 2
             
            #claculating the mistkae for the current case
            mistake += len(y_reqd) - ans
         #updating overall mistake   
        if mistake<least_mistake:
            least_mistake = mistake
            best_feature = f
            
            
    #Here calculating the count of 0,1,and 2 so as to get the gain ratio and entropy        
    possible_values = set(df[best_feature])
    entropy_after = 0
    arr2 =  []
    for val in possible_values:
        arr3 = []
        y_reqd = y[df[best_feature]==val]
        count_0_ = 0
        count_1_ = 0
        count_2_ = 0
        for  i in range(len(y_reqd)):
            if y_reqd.iloc[i,0] == 0:
                count_0_+=1
            elif y_reqd.iloc[i,0] == 1:
                count_1_+=1
            elif y_reqd.iloc[i,0] ==2:
                count_2_+=1
                
        arr3.append(count_0_)
        arr3.append(count_1_)
        arr3.append(count_2_)
        entropy_after+= entropy(arr3)
        arr2.append(len(y_reqd))
    
    split_info_after = split_info(arr2)
    
    #it is the gain ratio which is shown in this formula
    gain_ratio = (curr_entropy - entropy_after)/split_info_after  
    
    #printing as reqd
    print("Splitting on feature ", best_feature," with gain ratio " , gain_ratio)
    print(" ")

    # Now remove best feature from unused features
    unused_features.remove(best_feature)

    #loop over possible values of best feature
    possible_values = set(df[best_feature])
        
    for val in possible_values:
        # calling build tree recursively
        if(val == 'a'):#if val is 'a' then we have to link the current with that node
            curr_node.a = build_tree(df[df[best_feature]==val],y[df[best_feature]==val],unused_features,count+1)
        
        #similarly for b,c,d
        elif val == 'b':
            curr_node.b = build_tree(df[df[best_feature]==val],y[df[best_feature]==val],unused_features,count+1)
        
        elif val == 'c':
            curr_node.c = build_tree(df[df[best_feature]==val],y[df[best_feature]==val],unused_features,count+1)
        
        elif val == 'd':
            curr_node.d = build_tree(df[df[best_feature]==val],y[df[best_feature]==val],unused_features,count+1)
            
        
    return curr_node

In [213]:
y = pd.DataFrame(iris.target)
unused_features = set(df.columns)

root = build_tree(df, y, unused_features,0)
print("Printing using tree built in the function :")
print_tree(root,0)

LEVEL  0
count of 0 =  50
count of 1 =  50
count of 2 =  50
Current Entropy is = 1.0986122886681096
Splitting on feature  pw_labeled  with gain ratio  0.3999492084908657
 
LEVEL  1
count of 0 =  0
count of 1 =  10
count of 2 =  0
Current Entropy is = 0.0
Reached leaf node

LEVEL  1
count of 0 =  0
count of 1 =  0
count of 2 =  34
Current Entropy is = 0.0
Reached leaf node

LEVEL  1
count of 0 =  50
count of 1 =  0
count of 2 =  0
Current Entropy is = 0.0
Reached leaf node

LEVEL  1
count of 0 =  0
count of 1 =  40
count of 2 =  16
Current Entropy is = 0.5982695885852573
Splitting on feature  pl_labeled  with gain ratio  0.2858562221757157
 
LEVEL  2
count of 0 =  0
count of 1 =  1
count of 2 =  0
Current Entropy is = 0.0
Reached leaf node

LEVEL  2
count of 0 =  0
count of 1 =  0
count of 2 =  8
Current Entropy is = 0.0
Reached leaf node

LEVEL  2
count of 0 =  0
count of 1 =  39
count of 2 =  8
Current Entropy is = 0.45622342016761397
Splitting on feature  sl_labeled  with gain ratio 